# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)



#### Aluno Daniel Messias Santos 202110168
#### Aluno Thiago Pereira Freire 202110167

# Problema da Mochila Binária

O problema da mochila binária pode ser formulado da seguinte maneira:

## Função Objetivo

Maximizar o valor total da mochila:

$$
Z = \sum_{i=1}^{n} v_i \cdot x_i
$$

Onde:
- $ v_i $ é o valor do item $i$,
- $ x_i $ é uma variável binária que indica se o item $i$ é incluído na mochila ($x_i = 1$) ou não ($x_i = 0$).

## Restrição de Capacidade

A soma dos pesos dos itens selecionados não pode ultrapassar a capacidade da mochila:

$$
\sum_{i=1}^{n} w_i \cdot x_i \leq C
$$

Onde:
- $ w_i $ é o peso do item $i$,
- $ C $ é a capacidade total da mochila.

## Restrição de Binárias

As variáveis de decisão $x_i$ são binárias, ou seja:

$$
x_i \in \{0, 1\} \quad \text{para} \quad i = 1, 2, \dots, n
$$

## Relaxamento Fracionário (Upper Bound)

No relaxamento fracionário, as variáveis $x_i$ podem assumir valores contínuos entre 0 e 1, ou seja:

$$
0 \leq x_i \leq 1 \quad \text{para} \quad i = 1, 2, \dots, n
$$


#Implementação do algoritmo branch and bound

Seu código deve ser testado na seguinte instância: [link](https://drive.google.com/file/d/12CeZEow-6vVgJFgzXMo0gbjV5hLCM6Zi/view?usp=sharing). O README se encontra em: [link](https://drive.google.com/file/d/1LBTdkDoTQRxUJsKLI4Z38-Uc8oUPCZQ0/view?usp=sharing).

In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 83.9 MB/s eta 0:00:00


In [ ]:
import gurobipy as grb
import time

# Classe para representar um item da mochila
class Item:
    def __init__(self, peso, valor):
        self.peso = peso
        self.valor = valor
        self.ratio = valor / peso  # Razão valor/peso

# Função para resolver a mochila fracionária (como limite superior)
def mochila_fracionaria(capacidade, itens):
    # Ordenar os itens por razão valor/peso (decrescente)
    itens = sorted(itens, key=lambda x: x.ratio, reverse=True)

    valor_total = 0
    peso_total = 0

    for item in itens:
        if peso_total + item.peso <= capacidade:
            valor_total += item.valor
            peso_total += item.peso
        else:
            # Se não cabe todo o item, pega a fração que cabe
            remaining_capacity = capacidade - peso_total
            valor_total += item.valor * (remaining_capacity / item.peso)
            break

    return valor_total

# Função heurística gulosa para o limite inferior
def mochila_gulosa(capacidade, itens):
    # Ordenar os itens por razão valor/peso (decrescente)
    itens = sorted(itens, key=lambda x: x.ratio, reverse=True)

    valor_total = 0
    peso_total = 0

    for item in itens:
        if peso_total + item.peso <= capacidade:
            valor_total += item.valor
            peso_total += item.peso
        else:
            break

    return valor_total

# Função para resolver o problema da mochila binária usando Branch and Bound com Gurobi
def branch_and_bound(capacidade, itens):
    start_time = time.time()

    # Número de itens
    n = len(itens)

    # Inicializando o modelo Gurobi
    modelo = grb.Model()

    # Variáveis binárias: x_i = 1 se o item i for incluído, 0 caso contrário
    x = modelo.addVars(n, vtype=grb.GRB.BINARY, name="x")

    # Função objetivo: maximizar o valor total
    modelo.setObjective(grb.quicksum(itens[i].valor * x[i] for i in range(n)), grb.GRB.MAXIMIZE)

    # Restrição de capacidade da mochila
    modelo.addConstr(grb.quicksum(itens[i].peso * x[i] for i in range(n)) <= capacidade, "Capacidade")

    # Limite inferior (solução gulosa) e limite superior (solução fracionária)
    limite_inferior = mochila_gulosa(capacidade, itens)
    limite_superior = mochila_fracionaria(capacidade, itens)

    modelo.setParam('OutputFlag', 0)  # Desabilitar logs de solução no Gurobi

    # Resolver o modelo exato com Gurobi
    modelo.optimize()

    # Recuperar a melhor solução
    melhor_solucao = modelo.objVal

    end_time = time.time()
    tempo_computacional = end_time - start_time

    return melhor_solucao, limite_inferior, limite_superior, tempo_computacional

# Função para ler os dados de entrada a partir de um arquivo
def ler_entrada(arquivo):
    with open(arquivo, 'r') as f:
        nome_instancia = f.readline().strip()  # Linha 1: Nome da instância
        classe_instancia = f.readline().strip()  # Linha 2: Classe da instância
        capacidade = int(f.readline().strip())  # Linha 3: Capacidade da mochila

        itens = []

        # Leitura dos itens (peso e valor)
        for linha in f:
            if linha.strip():
                peso, valor = map(int, linha.split())
                itens.append(Item(peso, valor))

    return capacidade, itens

# Função principal para testar o algoritmo
def main():
    # Defina o caminho do arquivo de entrada
    arquivo_entrada = 'Weakly001'  # Substitua pelo caminho do seu arquivo de entrada

    capacidade, itens = ler_entrada(arquivo_entrada)

    melhor_solucao, limite_inferior, limite_superior, tempo_computacional = branch_and_bound(capacidade, itens)

    # Imprimir resultados
    print(f"Melhor solução encontrada: {melhor_solucao}")
    print(f"Limite inferior (heurística gulosa): {limite_inferior}")
    print(f"Limite superior (mochila fracionária): {limite_superior}")
    print(f"Tempo computacional: {tempo_computacional:.6f} segundos")

if __name__ == "__main__":
    main()


Restricted license - for non-production use only - expires 2026-11-23
Melhor solução encontrada: 4886.0
Limite inferior (heurística gulosa): 4886
Limite superior (mochila fracionária): 4887.475409836065
Tempo computacional: 0.028422 segundos
